## Create Index for `Past Conversations`

##### Prerequisites

In [ ]:
%%capture 

!pip install PyYAML

#### Imports

In [2]:
from requests.auth import HTTPBasicAuth
from tqdm import tqdm
import requests
import logging 
import boto3
import yaml
import json
import os

##### Setup logging

In [3]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies 

In [4]:
logger.info(f'Using requests=={requests.__version__}')
logger.info(f'Using pyyaml=={yaml.__version__}')

Using requests==2.28.2
Using pyyaml==6.0


#### Setup essentials

In [5]:
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)

os_username = config['credentials']['username']
os_password = config['credentials']['password']

domain_endpoint = config['domain']['endpoint']
domain_index = config['domain']['index']

In [6]:
URL = f'{domain_endpoint}/{domain_index}'
logger.info(f'URL for OpenSearch index = {URL}')

URL for OpenSearch index = https://search-semantic-search-hryn56c5jy43yryimohz4ajvyi.us-east-1.es.amazonaws.com/conversations


#### Define the index mapping with a k-NN vector field

In [7]:
mapping = {
    'settings': {
        'index': {
            'knn': True  # Enable k-NN search for this index
        }
    },
    'mappings': {
        'properties': {
            'embedding': {  # k-NN vector field
                'type': 'knn_vector',
                'dimension': 4096  # Dimension of the vector
            },
            'session_id': {
                'type': 'keyword'
            },
            'created_at': {
                'type': 'long'
            },
            'conversation_summary': {
                'type': 'text'
            }
        }
    }
}

#### Create the index with the specified mapping

In [8]:
# Check if the index exists using an HTTP HEAD request
response = requests.head(URL, auth=HTTPBasicAuth(os_username, os_password))

# If the index does not exist (status code 404), create the index
if response.status_code == 404:
    response = requests.put(URL, auth=HTTPBasicAuth(os_username, os_password), json=mapping)
    logger.info(f'Index created: {response.text}')
else:
    logger.error('Index already exists!')

Index created: {"acknowledged":true,"shards_acknowledged":true,"index":"conversations"}
